In [3]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime
import numpy as np
import timeit
import json

In [4]:
start_time = timeit.default_timer()


df_2019_july = pd.read_json('json_datasets/Kickstarter_2019-07-18T03_20_05_009Z.json', lines=True)


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

time: 147.19619212600003


In [5]:
# Save the creation datetime for calculating age of project later
df_2019_july_collected_on = datetime(2019, 7, 18)

In [6]:
# clean up the format and make a deep copy
start_time = timeit.default_timer()


df_2019_july = df_2019_july['data'].apply(pd.Series)
# df_2019_july_copy = df_2019_july.copy(deep=True)
print(f'Total_rows: {len(df_2019_july)}')


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

df_2019_july.head(3)

Total_rows: 212378
time: 129.102400075


,id,photo,name,blurb,goal,pledged,state,slug,disable_communication,country,currency,currency_symbol,currency_trailing_code,deadline,state_changed_at,created_at,launched_at,staff_pick,is_starrable,backers_count,static_usd_rate,usd_pledged,converted_pledged_amount,fx_rate,current_currency,usd_type,creator,location,category,profile,spotlight,urls,source_url,friends,is_starred,is_backing,permissions
0,1456501169,{'key': 'assets/014/965/917/1df84cc4df1ac7f377...,Citizen Carpentry Community Workshop & Tool Sh...,Citizen Carpentry is building community around...,14000.0,5528.0,canceled,citizen-carpentry-community-workshop-and-tool-...,False,US,USD,$,True,1487116800,1485878182,1479389359,1482855358,False,False,48,1.000000,5528.0,5528,1.000000,USD,domestic,"{'id': 2044486203, 'name': 'Marcis Curtis', 'i...","{'id': 2486982, 'name': 'St. Louis', 'slug': '...","{'id': 356, 'name': 'Woodworking', 'slug': 'cr...","{'id': 2763551, 'project_id': 2763551, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1,1259045207,{'key': 'assets/013/148/716/caff254d9e0c3fad79...,SIMBIN,El buscador sostenible para gente responsable,20000.0,0.0,failed,simbin,False,ES,EUR,€,False,1473524887,1473524887,1469258234,1469636887,False,False,0,1.098901,0.0,0,1.123945,USD,domestic,"{'id': 1498616534, 'name': 'Raül Gómez Freixa'...","{'id': 753692, 'name': 'Barcelona', 'slug': 'b...","{'id': 260, 'name': 'Interactive Design', 'slu...","{'id': 2610926, 'project_id': 2610926, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
2,577497800,{'key': 'assets/012/091/211/a030d9f7ce1ffe1011...,"Photographing Places I Love, Mountains of East...",Would love to show everyone the beauty Eastern...,5000.0,0.0,canceled,photographing-places-i-love-mountains-of-easte...,False,US,USD,$,True,1429640268,1429360155,1427808788,1428344268,False,False,0,1.000000,0.0,0,1.000000,USD,domestic,"{'id': 2019826059, 'name': 'Carmen', 'is_regis...","{'id': 2396395, 'name': 'Eastern', 'slug': 'ea...","{'id': 277, 'name': 'Nature', 'slug': 'photogr...","{'id': 1811349, 'project_id': 1811349, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


In [7]:
# # Check if data has same features
# df_2019_july_cols = df_2019_july.columns
# columns_original  = ['id', 'photo', 'name', 'blurb', 'goal', 'pledged', 'state', 'slug', 'disable_communication', 'country', 'country_displayable_name', 'currency', 'currency_symbol', 'currency_trailing_code', 'deadline', 'state_changed_at', 'created_at', 'launched_at', 'staff_pick', 'is_starrable', 'backers_count', 'static_usd_rate', 'usd_pledged', 'converted_pledged_amount', 'fx_rate', 'current_currency', 'usd_type', 'creator', 'location', 'category', 'profile', 'spotlight', 'urls', 'source_url', 'friends', 'is_starred', 'is_backing', 'permissions']
# diff = list(set(df_2019_july_cols) - set(columns_original))
# diff2 = list(set(columns_original) - set(df_2019_july_cols))
# print(diff)
# print(diff2)

In [8]:
# # inspect canceled and suspended states
# df_2019_july.state.value_counts()

In [9]:
# drop canceled and suspended states
df_2019_july.drop((df_2019_july[((df_2019_july.state == 'suspended') | 
                                         (df_2019_july.state == 'canceled'))].index), inplace=True)

# check for duplicates
print(f"Rows with duplicate project_id: {df_2019_july.duplicated(['id']).sum()}")

# drop duplicates
df_2019_july.drop_duplicates(['id'], inplace=True)
print(f'Total_rows after dropping duplicates and suspended/canceled states: {len(df_2019_july)}')


Rows with duplicate project_id: 27240
Total_rows after dropping duplicates and suspended/canceled states: 175847


In [10]:
# # get links to project and creator pages
# start_time = timeit.default_timer()


# df_2019_july['project_link'] = 'no_val'
# project_link_loc = df_2019_july.columns.get_loc('project_link')
# for n in range(len(df_2019_july)):
#     project_link = df_2019_july.iloc[n].urls['web']['project']
#     df_2019_july.iat[n, project_link_loc] = project_link
    
# df_2019_july['creator_link'] = 'no_val'
# creator_link_loc = df_2019_july.columns.get_loc('creator_link')
# for n in range(len(df_2019_july)):
#     creator_link = df_2019_july.iloc[n].creator['urls']['web']['user']
#     df_2019_july.iat[n, creator_link_loc] = creator_link  
    
    
# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

In [11]:
# # get main_category id
# start_time = timeit.default_timer()


# categories_map = {1:'Art',3:'Comics', 17:'Theater', 15:'Photography', 
#                 7:'Design', 6: 'Dance', 11:'Film & Video', 12:'Games', 16:'Technology', 
#                 13:'Journalism', 18:'Publishing', 9:'Fashion', 10:'Food', 14:'Music', 26:'Crafts'}
# categories_df = pd.DataFrame(categories_map, index=[0])


# df_2019_july['main_category'] = 'no_value'
# cat_loc = df_2019_july.columns.get_loc('main_category')


# for n in (range(len(df_2019_july))):
#     if 'parent_id' not in df_2019_july.iloc[n]['category'].keys():
#         cat_id = df_2019_july.iloc[n]['category']['id']
#         df_2019_july.iat[n, cat_loc] = cat_id
#     elif pd.isnull(df_2019_july.iloc[n]['category']['parent_id']):
#         cat_id = df_2019_july.iloc[n]['category']['id']
#         df_2019_july.iat[n, cat_loc] = cat_id
#     else:
#         par_id = df_2019_july.iloc[n]['category']['parent_id']
#         df_2019_july.iat[n, cat_loc] = par_id
        
# df_2019_july['main_category'] = df_2019_july['main_category'].apply(lambda x: categories_df[x])


# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

# df_2019_july.head(1)

In [12]:
# # keep these features
# df_main_feats = ['id', 'name', 'blurb', 'goal', 'pledged', 'state', 'country',
# 'currency', 'deadline', 'launched_at', 'staff_pick', 'main_category',
# 'backers_count', 'static_usd_rate', 'usd_pledged' ]
# # Drop these 
# df_drop_feats = list(set(df_2019_july_cols) - set(df_main_feats))
# print(f'Columns removed: {df_drop_feats}')

# # drop features
# df_2019_july.drop(df_drop_feats, axis=1, inplace=True)

In [13]:
# convert unix dates to datetime objects
df_2019_july.deadline = df_2019_july.deadline.apply(lambda x: (datetime.fromtimestamp(x)))
df_2019_july.launched_at = df_2019_july.launched_at.apply(lambda x: (datetime.fromtimestamp(x)))

In [14]:
# rename columns
df_2019_july.rename(columns={'id':'project_id'}, inplace=True)
df_2019_july.rename(columns={'name':'project_name'}, inplace=True)
df_2019_july.rename(columns={'blurb':'project_description'}, inplace=True)
df_2019_july.rename(columns={'state':'outcome_state'}, inplace=True)


In [15]:
# add project duration feature
df_2019_july['duration'] = df_2019_july.deadline - df_2019_july.launched_at
df_2019_july['days_from_launch'] =(df_2019_july_collected_on - df_2019_july.launched_at).apply(lambda x: x.days)

# # add feature 'in_mid_duration_range' indicating whether project is about halfway through
# df_2019_july['duration_mid_point'] = df_2019_july.duration.apply(lambda x: x.days) / 2
# df_2019_july['lower_bound_mid_point_range'] = df_2019_july.duration_mid_point.apply(lambda x: x - 5)
# df_2019_july['upper_bound_mid_point_range'] = df_2019_july.duration_mid_point.apply(lambda x: x + 5)
# df_2019_july['in_mid_duration_range'] = df_2019_july.days_from_launch.between(df_2019_july.lower_bound_mid_point_range, df_2019_july.upper_bound_mid_point_range)

# df_2019_july.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)


In [16]:
# df_2019_july['duration'][0].quantile(.1)
df_2019_july['duration_10_percent'] = df_2019_july.duration.apply(lambda x: x.days / 10 ) 


In [17]:
# add feature 'in_mid_duration_range' indicating whether project is about halfway through
df_2019_july['lower_bound'] = df_2019_july.duration_10_percent.apply(lambda x: x if x < 5 else x - 5)
df_2019_july['upper_bound'] = df_2019_july.duration_10_percent.apply(lambda x: x + 5)
df_2019_july['in_range'] = df_2019_july.days_from_launch.between(df_2019_july.lower_bound, df_2019_july.upper_bound)

# df_2019_july.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)

In [18]:
# calculate usd_goal
df_2019_july['usd_goal'] = df_2019_july.goal * df_2019_july.static_usd_rate


In [19]:
# Replace goal and rearrange
# col_list = df_2019_july.columns.to_list()
# print(col_list)
col_list_rearrange = ['project_id', 'project_name', 'project_description', 'outcome_state', 'in_range',
                      'duration_10_percent', 'days_from_launch', 
                      'usd_goal', 'usd_pledged', 'currency', 'launched_at', 'deadline', 'staff_pick', 'country', 
                      'duration' , 'backers_count']
df_2019_july = df_2019_july[col_list_rearrange]
df_2019_july.head(1)
                      

,project_id,project_name,project_description,outcome_state,in_range,duration_10_percent,days_from_launch,usd_goal,usd_pledged,currency,launched_at,deadline,staff_pick,country,duration,backers_count
1,1259045207,SIMBIN,El buscador sostenible para gente responsable,failed,False,4.5,1085,21978.022,0.0,EUR,2016-07-27 12:28:07,2016-09-10 12:28:07,False,ES,45 days,0


In [20]:
df_2019_july_copy = df_2019_july.copy(deep=True)
# convert datetime columns before exporting
# df_2019_july.duration = df_2019_july.duration.apply(lambda x: x.days) 
df_2019_july.launched_at = df_2019_july.launched_at.apply(lambda x: x.strftime('%m-%d-%Y'))
df_2019_july.deadline = df_2019_july.deadline.apply(lambda x: x.strftime('%m-%d-%Y'))

In [21]:
# export cleaned dataframe to csv
df_2019_july.to_csv('clean_datasets_10/main_df_2019_july.csv', index=False)